In [1]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
#Lets provide some insight into PO.DAAC impact metrics 
#in Elseviers SCOPUS service

In [3]:
#First lets import the libraries we require
from pprint import pprint
import podaac.podaac as podaac
import podaac.podaac_utils as utils

In [4]:
#Then we can create instances of the classes we will use
p = podaac.Podaac()
u = utils.PodaacUtils()

In [5]:
#Print a list of dataset short names
print('\nHeres list_all_available_granule_search_dataset_short_names()')
result = u.list_all_available_granule_search_dataset_short_names()
dsetShortName = [i for i in result]
print(dsetShortName)
print(len(result))


Heres list_all_available_granule_search_dataset_short_names()
['CCMP_MEASURES_ATLAS_L3_OW_L2_5_SSMI_F8_WIND_VECTORS_FLK', 'TELLUS_OCEANEOF_ANIM_GFZ_RL05', 'VIIRS_NPP-OSPO-L3U-v2.41', 'TOPEX_L2_OST_TMR_Replacement', 'MODIS_TERRA_L3_SST_THERMAL_8DAY_9KM_DAYTIME_V2014.0', 'AQUARIUS_L3_SPICINESS_SMID_MONTHLY_V5', 'SEAWINDS_BYU_L3_OW_SIGMA0_ARCTIC_POLAR-STEREOGRAPHIC_BROWSE_IMAGES_LTOD', 'MERGED_TP_J1_OSTM_OST_ALL_V42', 'QSCAT_BYU_L3_OW_SIGMA0_ANTARCTICA_POLAR-STEREOGRAPHIC_BROWSE_IMAGES', 'AQUARIUS_L3_SSS_SMI_DAILY_V5', 'UKMO-L4HRfnd-GLOB-OSTIA', 'GRACE_GAB_L2_GRAV_GFZ_RL05_WEEK', 'TMI-REMSS-L2P-v4', 'Geo_Polar_Blended_Night-OSPO-L4-GLOB-v1.0', 'AQUARIUS_L3_SPICINESS_SMID_CUMULATIVE_V5', 'TELLUS_OCEANEOF_ANIM_JPL_RL05', 'REMSS-L2P_GRIDDED_25-WSAT', 'JASON-1_L2_OST_GPN_E_GEODETIC', 'CYGNSS_L2_V2.1', 'SPURS1_METEO', 'WindSat-REMSS-L3U-v7.0.1a', 'GOES13-OSISAF-L3C-v1.0', 'TELLUS_LAND_GIF_GFZ_RL05', 'TELLUS_LAND_GTIF_GFZ_RL05', 'AQUARIUS_L3_ANCILLARY_SST_SMI_SEASONAL-CLIMATOLOGY_V5', 'NSCAT_2

In [6]:
#dataset_landing_pages = []
#Perform a landing page search on each dataset, note this can take some time as we are querying >600 times
#for ds in dsetShortName:
#    result = p.dataset_search(short_name=ds)
#    #Cache the dataset landing page URL
#    searchStr = 'http://podaac.jpl.nasa.gov/dataset/'
#    for i in result.strip().split():
#        if searchStr in i:
#            dataset_landing_pages.append(str(i))
#print(dataset_landing_pages)

In [7]:
from bs4 import BeautifulSoup
dois = []
#Perform a doi search on each dataset, note this can take some time as we are querying >600 times
for ds in dsetShortName:
    try:
        result = p.dataset_search(short_name=ds, full='True')
    except Exception:
        print('Error retrieving record for ' + ds)
    soup = BeautifulSoup(result, 'html.parser')
    doi_element = soup.find('podaac:datasetdoi', text=True)
    if doi_element is not None:
        doi = doi_element.text #do not include 'https://doi.org/' prefix as it eventually makes search less accurate and increases runtime
        dois.append(doi)
print(dois)

404 Client Error: Not Found for url: https://podaac.jpl.nasa.gov/ws/search/dataset/?itemsPerPage=7&pretty=True&format=atom&full=True&shortName=WENTZ_SASS_SIGMA0_L2
Error retrieving record for WENTZ_SASS_SIGMA0_L2
404 Client Error: Not Found for url: https://podaac.jpl.nasa.gov/ws/search/dataset/?itemsPerPage=7&pretty=True&format=atom&full=True&shortName=UCLA_DEALIASED_SASS_L3
Error retrieving record for UCLA_DEALIASED_SASS_L3
404 Client Error: Not Found for url: https://podaac.jpl.nasa.gov/ws/search/dataset/?itemsPerPage=7&pretty=True&format=atom&full=True&shortName=ATLAS_DEALIASED_SASS_L2
Error retrieving record for ATLAS_DEALIASED_SASS_L2
404 Client Error: Not Found for url: https://podaac.jpl.nasa.gov/ws/search/dataset/?itemsPerPage=7&pretty=True&format=atom&full=True&shortName=SEASAT_BYU_L3_OW_SIGMA0_ENHANCED
Error retrieving record for SEASAT_BYU_L3_OW_SIGMA0_ENHANCED
404 Client Error: Not Found for url: https://podaac.jpl.nasa.gov/ws/search/dataset/?itemsPerPage=7&pretty=True&for

In [8]:
print(len(dois))

568


In [21]:
import json
import requests
data = []
#Using Elsevier's Scopus Search, lets see if we can 
#retreieve any information from the above dataset dois
for doi in dois:
    url = 'https://api.elsevier.com/content/search/scopus?query=ALL:' + doi + '&APIKey=715b412c00f0b95e918a3e7abe6e6ee4'
    try:
        metadata = requests.get(url)
        status_codes = [404, 400, 503, 408]
        if metadata.status_code in status_codes:
            metadata.raise_for_status()
    except requests.exceptions.HTTPError as error:
        print(error)
        raise
    json_data = json.loads(metadata.text)
    entries = json_data['search-results']['entry']
    for entry in entries:
        if 'error' not in entry:
            print(entry)
            try:
                prism_doi = entry['prism:doi']
            except KeyError:
                prism_doi = '.'               
            data.append(['.', '.', entry['dc:title'], entry['prism:publicationName'], entry['citedby-count'], entry['prism:url'], prism_doi, '.', '.', '.', '.', '.', '.', '.', '.'])

{'@_fa': 'true', 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85050387237'}, {'@_fa': 'true', '@ref': 'author-affiliation', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85050387237?field=author,affiliation'}, {'@_fa': 'true', '@ref': 'scopus', '@href': 'https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85050387237&origin=inward'}, {'@_fa': 'true', '@ref': 'scopus-citedby', '@href': 'https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85050387237&origin=inward'}, {'@_fa': 'true', '@ref': 'full-text', '@href': 'https://api.elsevier.com/content/article/eid/1-s2.0-S135943111737415X'}], 'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/85050387237', 'dc:identifier': 'SCOPUS_ID:85050387237', 'eid': '2-s2.0-85050387237', 'dc:title': 'Evaluation of satellite-derived SST products in identifying the rapid temperature drop on the West Florida shelf associated with Hurricane Irma'

{'@_fa': 'true', 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85055466337'}, {'@_fa': 'true', '@ref': 'author-affiliation', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85055466337?field=author,affiliation'}, {'@_fa': 'true', '@ref': 'scopus', '@href': 'https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85055466337&origin=inward'}, {'@_fa': 'true', '@ref': 'scopus-citedby', '@href': 'https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85055466337&origin=inward'}], 'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/85055466337', 'dc:identifier': 'SCOPUS_ID:85055466337', 'eid': '2-s2.0-85055466337', 'dc:title': 'Physical modulation to the biological productivity in the summer Vietnam upwelling system', 'dc:creator': 'Lu W.', 'prism:publicationName': 'Ocean Science', 'prism:issn': '18120784', 'prism:eIssn': '18120792', 'prism:volume': '14', 'prism:issueIdentifier': '5', 'pr

{'@_fa': 'true', 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85021805979'}, {'@_fa': 'true', '@ref': 'author-affiliation', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85021805979?field=author,affiliation'}, {'@_fa': 'true', '@ref': 'scopus', '@href': 'https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85021805979&origin=inward'}, {'@_fa': 'true', '@ref': 'scopus-citedby', '@href': 'https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85021805979&origin=inward'}], 'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/85021805979', 'dc:identifier': 'SCOPUS_ID:85021805979', 'eid': '2-s2.0-85021805979', 'dc:title': 'Movements and foraging habitats of great shearwaters Puffinus gravis in the Gulf of Maine', 'dc:creator': 'Powers K.', 'prism:publicationName': 'Marine Ecology Progress Series', 'prism:issn': '01718630', 'prism:volume': '574', 'prism:pageRange': '211-226', 'prism:c

{'@_fa': 'true', 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/84959520742'}, {'@_fa': 'true', '@ref': 'author-affiliation', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/84959520742?field=author,affiliation'}, {'@_fa': 'true', '@ref': 'scopus', '@href': 'https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=84959520742&origin=inward'}, {'@_fa': 'true', '@ref': 'scopus-citedby', '@href': 'https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=84959520742&origin=inward'}], 'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/84959520742', 'dc:identifier': 'SCOPUS_ID:84959520742', 'eid': '2-s2.0-84959520742', 'dc:title': 'Sea breeze characteristics on two sides of a shallow gulf: Study of the Gulf St Vincent in South Australia', 'dc:creator': 'Pazandeh Masouleh Z.', 'prism:publicationName': 'Meteorological Applications', 'prism:issn': '13504827', 'prism:eIssn': '14698080', 'prism:vo

{'@_fa': 'true', 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85058517073'}, {'@_fa': 'true', '@ref': 'author-affiliation', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85058517073?field=author,affiliation'}, {'@_fa': 'true', '@ref': 'scopus', '@href': 'https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85058517073&origin=inward'}, {'@_fa': 'true', '@ref': 'scopus-citedby', '@href': 'https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85058517073&origin=inward'}], 'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/85058517073', 'dc:identifier': 'SCOPUS_ID:85058517073', 'eid': '2-s2.0-85058517073', 'dc:title': 'The efficacy of aerosol-cloud radiative perturbations from near-surface emissions in deep open-cell stratocumuli', 'dc:creator': 'Possner A.', 'prism:publicationName': 'Atmospheric Chemistry and Physics', 'prism:issn': '16807316', 'prism:eIssn': '16807324', 'prism:

{'@_fa': 'true', 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85050387237'}, {'@_fa': 'true', '@ref': 'author-affiliation', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85050387237?field=author,affiliation'}, {'@_fa': 'true', '@ref': 'scopus', '@href': 'https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85050387237&origin=inward'}, {'@_fa': 'true', '@ref': 'scopus-citedby', '@href': 'https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85050387237&origin=inward'}, {'@_fa': 'true', '@ref': 'full-text', '@href': 'https://api.elsevier.com/content/article/eid/1-s2.0-S135943111737415X'}], 'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/85050387237', 'dc:identifier': 'SCOPUS_ID:85050387237', 'eid': '2-s2.0-85050387237', 'dc:title': 'Evaluation of satellite-derived SST products in identifying the rapid temperature drop on the West Florida shelf associated with Hurricane Irma'

{'@_fa': 'true', 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85057122740'}, {'@_fa': 'true', '@ref': 'author-affiliation', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85057122740?field=author,affiliation'}, {'@_fa': 'true', '@ref': 'scopus', '@href': 'https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85057122740&origin=inward'}, {'@_fa': 'true', '@ref': 'scopus-citedby', '@href': 'https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85057122740&origin=inward'}], 'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/85057122740', 'dc:identifier': 'SCOPUS_ID:85057122740', 'eid': '2-s2.0-85057122740', 'dc:title': 'Impact of the elevation angle on CYGNSS GNSS-R bistatic reflectivity as a function of effective surface roughness over land surfaces', 'dc:creator': 'Carreno-Luengo H.', 'prism:publicationName': 'Remote Sensing', 'prism:eIssn': '20724292', 'prism:volume': '10', 'pr

{'@_fa': 'true', 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/84994289066'}, {'@_fa': 'true', '@ref': 'author-affiliation', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/84994289066?field=author,affiliation'}, {'@_fa': 'true', '@ref': 'scopus', '@href': 'https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=84994289066&origin=inward'}, {'@_fa': 'true', '@ref': 'scopus-citedby', '@href': 'https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=84994289066&origin=inward'}], 'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/84994289066', 'dc:identifier': 'SCOPUS_ID:84994289066', 'eid': '2-s2.0-84994289066', 'dc:title': 'Evaluation of GRACE Mascon Gravity Solution in Relation to Interannual Oceanic Water Mass Variations', 'dc:creator': 'Melzer B.', 'prism:publicationName': 'IEEE Transactions on Geoscience and Remote Sensing', 'prism:issn': '01962892', 'prism:volume': '55', 'prism:i

{'@_fa': 'true', 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85050767239'}, {'@_fa': 'true', '@ref': 'author-affiliation', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85050767239?field=author,affiliation'}, {'@_fa': 'true', '@ref': 'scopus', '@href': 'https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85050767239&origin=inward'}, {'@_fa': 'true', '@ref': 'scopus-citedby', '@href': 'https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85050767239&origin=inward'}], 'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/85050767239', 'dc:identifier': 'SCOPUS_ID:85050767239', 'eid': '2-s2.0-85050767239', 'dc:title': 'Analyzing Large-Scale Hydrologic Processes Using GRACE and Hydrometeorological Datasets', 'dc:creator': 'Banerjee C.', 'prism:publicationName': 'Water Resources Management', 'prism:issn': '09204741', 'prism:eIssn': '15731650', 'prism:volume': '32', 'prism:issueIden

{'@_fa': 'true', 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85050944547'}, {'@_fa': 'true', '@ref': 'author-affiliation', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85050944547?field=author,affiliation'}, {'@_fa': 'true', '@ref': 'scopus', '@href': 'https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85050944547&origin=inward'}, {'@_fa': 'true', '@ref': 'scopus-citedby', '@href': 'https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85050944547&origin=inward'}], 'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/85050944547', 'dc:identifier': 'SCOPUS_ID:85050944547', 'eid': '2-s2.0-85050944547', 'dc:title': 'Understanding terrestrial water storage variations in northern latitudes across scales', 'dc:creator': 'Trautmann T.', 'prism:publicationName': 'Hydrology and Earth System Sciences', 'prism:issn': '10275606', 'prism:eIssn': '16077938', 'prism:volume': '22', 'prism:

In [25]:
#Again, using Elsevier's Scopus Search, lets see other podaac.jpl.nasa.gov resource we can retreive.
url = 'https://api.elsevier.com/content/search/scopus?query=ALL:podaac.jpl.nasa.gov&APIKey=715b412c00f0b95e918a3e7abe6e6ee4'
try:
    metadata = requests.get(url)
    status_codes = [404, 400, 503, 408]
    if metadata.status_code in status_codes:
        metadata.raise_for_status()
except requests.exceptions.HTTPError as error:
    print(error)
    raise
json_data = json.loads(metadata.text)
entries = json_data['search-results']['entry']
for entry in entries:
    data.append(['.', '.', entry['dc:title'], entry['prism:publicationName'], entry['citedby-count'], entry['prism:url'], entry['prism:doi'], '.', '.', '.', '.', '.', '.', '.', '.'])

In [26]:
import pandas as pd
#Simply print the data as a pandas data frame
pd.DataFrame(data, columns=["Dataset Name as Cited in Article", "PERSISTENT_ID", "Manuscript Title", "Journal Source", "Impact factor of the Manuscript", "Article URL", "DOI", "PDF Name", "In-Text", "In-Text 2", "Referenced", "Referenced 2", "In-Text", "Referenced", "Confirmed"])

,Dataset Name as Cited in Article,PERSISTENT_ID,Manuscript Title,Journal Source,Impact factor of the Manuscript,Article URL,DOI,PDF Name,In-Text,In-Text 2,Referenced,Referenced 2,In-Text,Referenced,Confirmed
0,.,.,Evaluation of satellite-derived SST products i...,Marine Technology Society Journal,2,https://api.elsevier.com/content/abstract/scop...,10.4031/MTSJ.52.3.7,.,.,.,.,.,.,.,.
1,.,.,Revealing the full ocean migration of individu...,Animal Biotelemetry,2,https://api.elsevier.com/content/abstract/scop...,10.1186/s40317-018-0146-2,.,.,.,.,.,.,.,.
2,.,.,Impact of SST on heavy rainfall events on east...,Atmospheric Research,2,https://api.elsevier.com/content/abstract/scop...,10.1016/j.atmosres.2017.09.019,.,.,.,.,.,.,.,.
3,.,.,Impact of land-cover change between 1990 and 2...,Hydrological Research Letters,0,https://api.elsevier.com/content/abstract/scop...,10.3178/hrl.11.187,.,.,.,.,.,.,.,.
4,.,.,Full-flow-regime storage-streamflow correlatio...,Water Resources Research,5,https://api.elsevier.com/content/abstract/scop...,10.1002/2016WR020283,.,.,.,.,.,.,.,.
5,.,.,An economist’s guide to climate change Science,Journal of Economic Perspectives,0,https://api.elsevier.com/content/abstract/scop...,10.1257/jep.32.4.3,.,.,.,.,.,.,.,.
6,.,.,Physical modulation to the biological producti...,Ocean Science,0,https://api.elsevier.com/content/abstract/scop...,10.5194/os-14-1303-2018,.,.,.,.,.,.,.,.
7,.,.,Review of energy systems deployment and develo...,Energy,2,https://api.elsevier.com/content/abstract/scop...,10.1016/j.energy.2018.07.185,.,.,.,.,.,.,.,.
8,.,.,Anchovy (Engraulis encrasicolus) early life st...,Hydrobiologia,3,https://api.elsevier.com/content/abstract/scop...,10.1007/s10750-017-3253-9,.,.,.,.,.,.,.,.
9,.,.,Uncertainty in the global oceanic CO2 uptake i...,Biogeosciences,0,https://api.elsevier.com/content/abstract/scop...,10.5194/bg-15-1701-2018,.,.,.,.,.,.,.,.
